In [1]:
%load_ext autoreload
%autoreload 2

import alpaca_trade_api
import lib.Broker as Broker
import asyncio
import lib.TimeKeeper as tk
import datetime
import pytz
import Toolbox as tb
from alpaca_trade_api.common import URL
from alpaca_trade_api.stream import Stream
import pandas as pd
import nest_asyncio
nest_asyncio.apply()
%autoawait asyncio

In [58]:
tb.get_data('NVDA', '1m', '1d')

,open,high,low,close,volume,dividends,stock splits
Datetime,,,,,,,
2024-03-20 09:30:00-04:00,898.900024,902.369873,894.260071,900.681824,1833664,0.0,0.0
2024-03-20 09:31:00-04:00,900.599976,903.219971,896.969971,903.219971,346208,0.0,0.0
2024-03-20 09:32:00-04:00,903.219971,904.099976,899.506226,900.999878,305957,0.0,0.0
2024-03-20 09:33:00-04:00,900.715027,901.399902,897.840027,900.950012,242337,0.0,0.0
2024-03-20 09:34:00-04:00,900.833923,902.700012,898.030090,899.109985,254816,0.0,0.0
...,...,...,...,...,...,...,...
2024-03-20 14:07:00-04:00,897.229980,897.809998,891.380005,891.380005,248700,0.0,0.0
2024-03-20 14:08:00-04:00,891.541809,894.719971,891.541809,894.287903,212418,0.0,0.0
2024-03-20 14:09:00-04:00,894.354980,895.189880,890.570007,891.443115,175324,0.0,0.0


In [38]:
api = Broker.paper_api()
#print(dir(alpaca_trade_api.TimeFrame.Minute))
s = tk.to_time(3,19,13,12)
e = tk.to_time(3,19,13,13)
df = api.get_bars('NVDA', alpaca_trade_api.TimeFrame.Minute).df#, start=s, end=e).df
df.index = df.index.tz_convert(pytz.timezone('US/Eastern'))
print(df)

Activating with Paper
                              close      high       low  trade_count  \
timestamp                                                              
2024-03-20 04:00:00-04:00  892.0000  894.1000  890.2200          162   
2024-03-20 04:01:00-04:00  891.8000  892.1300  891.0100          130   
2024-03-20 04:02:00-04:00  891.8000  891.8000  891.8000           85   
2024-03-20 04:03:00-04:00  892.0000  892.0000  892.0000          103   
2024-03-20 04:04:00-04:00  893.5000  893.5000  893.5000          106   
...                             ...       ...       ...          ...   
2024-03-20 14:01:00-04:00  896.5765  897.4499  891.2000         8002   
2024-03-20 14:02:00-04:00  893.2450  898.0000  892.6801         5766   
2024-03-20 14:03:00-04:00  895.5800  895.7700  892.2200         3417   
2024-03-20 14:04:00-04:00  896.1100  896.8000  894.0000         3638   
2024-03-20 14:05:00-04:00  897.9699  898.9000  896.0000         5620   

                               open  volu

# Things We Must Do
1. Implement Async into Trader
2. Figure out how to get bars up to date
3. Implement async

In [ ]:
from alpaca_trade_api.rest import REST, TimeFrame
import lib.Broker as Broker
api = Broker.live_api()
#
#api.get_bars("NVDA", TimeFrame.Minute, asof='2024-03-20', adjustment='raw').df
#api.get_latest_bars(['AMZN','NVDA'])
#print(api.get_latest_bar('AMZN'))

In [70]:
bars = api.get_bars(['AMZN'], TimeFrame.Minute, feed='sip').df
print(bars.sort_index())

                            close    high      low  trade_count     open  \
timestamp                                                                  
2024-03-20 08:01:00+00:00  175.77  175.77  175.770           35  175.770   
2024-03-20 08:04:00+00:00  175.57  175.57  175.570           10  175.570   
2024-03-20 08:07:00+00:00  175.62  175.62  175.620           15  175.620   
2024-03-20 08:13:00+00:00  175.57  175.57  175.570            5  175.570   
2024-03-20 08:17:00+00:00  175.62  175.62  175.620           13  175.620   
...                           ...     ...      ...          ...      ...   
2024-03-20 13:25:00+00:00  176.00  176.00  175.840           62  175.840   
2024-03-20 13:26:00+00:00  176.02  176.07  176.010           47  176.030   
2024-03-20 13:27:00+00:00  176.13  176.13  176.063           70  176.063   
2024-03-20 13:28:00+00:00  176.19  176.19  176.100           59  176.100   
2024-03-20 13:29:00+00:00  176.13  176.13  176.110           58  176.110   

           

# What Must we Learn Here?
We must learn how this async stream library works:
1. We must learn whether or not the information it gives is credible
2. We must learn whether the information it gives matches up with the bars
3. We must implement it so that we can use it to trade


Before we had a time based approach, now we must move to an event based approach. The bars will mean that it's almost a time based approach, so for example, we might say "if bar's timestamp is after 3:50, cashout" but for the most part we need to reframe everything around event calls

In [ ]:
api.get_bars('NVDA', alpaca_trade_api.TimeFrame.Minute).df

In [3]:
import nest_asyncio
nest_asyncio.apply()

# async def trade_callback(t):
#     '''
#         for ticker in self.tickers:
#             ticker.record_trade(t)
#     '''
#     print('trade', t)

async def bar_callback(b):
    data.append(b)
    print(tk.now(),'IS THE TIME')
    '''
        For ticker in self.tickers:
            ticker.trade_command(b)
    
    '''
    print(api.get_bars('NVDA', alpaca_trade_api.TimeFrame.Minute).df)
    print('bar', pd.to_datetime(b.timestamp).tz_localize(pytz.UTC).tz_convert(pytz.timezone('US/Eastern')))

# stream.subscribe_trades(trade_callback, 'NVDA')
stream.subscribe_bars(bar_callback, 'NVDA')
stream.run()

2024-03-20 15:16:00-04:00 IS THE TIME
                              close      high       low  trade_count    open  \
timestamp                                                                      
2024-03-20 08:00:00+00:00  892.0000  894.1000  890.2200          162  890.22   
2024-03-20 08:01:00+00:00  891.8000  892.1300  891.0100          130  892.13   
2024-03-20 08:02:00+00:00  891.8000  891.8000  891.8000           85  891.80   
2024-03-20 08:03:00+00:00  892.0000  892.0000  892.0000          103  892.00   
2024-03-20 08:04:00+00:00  893.5000  893.5000  893.5000          106  893.50   
...                             ...       ...       ...          ...     ...   
2024-03-20 19:11:00+00:00  899.0199  901.0000  898.7064         2290  900.25   
2024-03-20 19:12:00+00:00  898.4992  899.5499  898.0201         2241  898.59   
2024-03-20 19:13:00+00:00  898.2750  899.1400  898.0000         2880  898.27   
2024-03-20 19:14:00+00:00  898.8849  899.5900  898.0000         2048  898.29   
20

In [23]:
import datetime
pd.to_datetime(data[0].timestamp).tz_localize(pytz.UTC).tz_convert(pytz.timezone('America/New_York'))
#print(data[0].timestamp)
#print(pd.to_datetime(int(data[0].timestamp), utc=True, unit='ms'))
#datetime.datetime.fromtimestamp(data[0].timestamp, tz=pytz.UTC)

Timestamp('2024-03-20 14:23:00-0400', tz='America/New_York')

In [ ]:
stream.on_bar('NVDA')

In [ ]:
stream.stop()

In [ ]:
data

In [ ]:
simplified_data = []
for d in data:
    simplified_data.append({'index':d['index'], 'price':d['trade'].price, 'size':d['trade'].size, 'timestamp':d['trade'].timestamp, 'symbol':d['trade'].symbol})
df = pd.DataFrame(simplified_data).set_index('timestamp')
#df.price.plot.line()
for l in df.index:
    print(l, '\t',  df.loc[l].price)

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df.to_pickle('./archive_data/_trade_data/nvda_trades_march_19.pkl')

In [ ]:
a = pd.read_pickle('./archive_data/_trade_data/nvda_trades_march_19.pkl')